In [1]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
import struct
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    print("===============================")
    
    # Overlay
    ol = Overlay("fiFFNTT.bit")
    ipFN = ol.fiFFNTT_0

    # Memory allocate
    
    Buffer0 = allocate(shape=(1024,), dtype=np.uint64)
    Buffer1 = allocate(shape=(1024,), dtype=np.uint64)
    Buffer2 = allocate(shape=(1024,), dtype=np.float64)
    Buffer3 = allocate(shape=(1024,), dtype=np.float64)
    
    ntt_out  = [0 for i in range(1024)]
    intt_out = [0 for i in range(1024)]
    fft_out  = [0 for i in range(1024)]
    ifft_out = [0 for i in range(1024)]
    
    # ==================== NTT ========================
    # =============== NTT test vector =================
    ntt_time = 0
    for ntt_test in range(5):
        with open("mq_ntt_in.txt", "r") as NTT_in:
            for i in range(1024):
                Buffer0[i] = np.uint16(NTT_in.readline())
        #print(bin(Buffer0[0]))

        timeKernelStart = time()

        mode = 0            # NTT_req
        ipFN.write(0x1C, 0) # mode    = NTT
        ipFN.write(0x24, 0) # inverse = false
        ipFN.write(0x10, Buffer0.device_address)

    # ================= Start NTT =====================

        ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipFN.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        ntt_time += (timeKernelEnd - timeKernelStart)
    for i in range(1024):
        ntt_out[i] = Buffer0[i]
        
    print("Kernel execution time [mode: NTT]: " + str(ntt_time/5) + " s")
    # =================================================
    
    
    # =================== iNTT ========================
    # ============== iNTT test vector =================
    intt_time=0
    for intt_test in range(5):
        with open("mq_ntt_out.txt", "r") as iNTT_in:
            for i in range(1024):
                Buffer1[i] = np.uint16(iNTT_in.readline())
        #print(bin(Buffer1[0]))

        timeKernelStart = time()

        ipFN.write(0x1C, 0) # mode    = NTT
        ipFN.write(0x24, 1) # inverse = true
        ipFN.write(0x10, Buffer1.device_address)

    # ================ Start iNTT =====================

        ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipFN.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        intt_time += (timeKernelEnd - timeKernelStart)

    for i in range(1024):
        intt_out[i] = Buffer1[i]
        
    print("Kernel execution time [mode: iNTT]: " + str(intt_time/5) + " s")
    # =================================================
    
    
    # ==================== FFT ========================
    # =============== FFT test vector =================
    fft_time = 0
    for fft_test in range(5):
        with open("FFT_in.txt", "r") as FFT_in:
            for i in range(1024):
                Buffer2[i] = float(FFT_in.readline())

        timeKernelStart = time()
        ipFN.write(0x1C, 1) # mode    = FFT
        ipFN.write(0x24, 0) # inverse = false
        ipFN.write(0x10, Buffer2.device_address)

    # ================= Start FFT =====================

        ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipFN.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        fft_time += (timeKernelEnd - timeKernelStart)

    for i in range(1024):
        fft_out[i] = Buffer2[i]
        
    print("Kernel execution time [mode: FFT]: " + str(fft_time/5) + " s")
    # =================================================

    
    # =================== iFFT ========================
    # ============== iFFT test vector =================
    ifft_time = 0
    for ifft_test in range(5):
        with open("FFT_out.txt", "r") as iFFT_in:
            for i in range(1024):
                Buffer3[i] = float(iFFT_in.readline())
        timeKernelStart = time()
        
        mode = 0            # FFT_req
        ipFN.write(0x1C, 1) # mode    = FFT
        ipFN.write(0x24, 1) # inverse = true
        ipFN.write(0x10, Buffer3.device_address)
    
    # ================ Start iFFT =====================
        ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipFN.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        ifft_time += (timeKernelEnd - timeKernelStart)

    for i in range(1024):
        ifft_out[i] = Buffer3[i]
        
    print("Kernel execution time [mode: iFFT]: " + str(ifft_time/5) + " s")
    # =================================================
    
    # ================== Compare ======================
    print("-------------------------------")
    print("Compare NTT with golden vector:")
    error_cnt = 0
    with open("mq_ntt_out.txt", "r") as NTT_out:
        for i in range(1024):
            golden = np.uint16(NTT_out.readline())
            if ntt_out[i] != golden:
                print(f"{i:<4} Failed!! Expect value: {golden}. Actual value: {ntt_out[i]}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
        
        
    print("-------------------------------")
    print("Compare iNTT with golden vector:")
    error_cnt = 0
    with open("mq_ntt_in.txt", "r") as iNTT_out:
        for i in range(1024):
            golden = np.uint16(iNTT_out.readline())
            if intt_out[i] != golden:
                print(f"{i:<4} Failed!! Expect value: {golden}. Actual value: {intt_out[i]}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    
    
    print("-------------------------------")
    print("Compare FFT with golden vector:")
    error_cnt = 0
    round_factor = 6
    with open("FFT_out.txt", "r") as FFT_out:
        for i in range(1024):
            golden = float(FFT_out.readline())
            if round(fft_out[i], round_factor) != round(golden, round_factor):
                print(f"{i:<4}Failed!! Expect value: {golden:.17f}. Actual value: {fft_out[i]:.17f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    
    
    print("-------------------------------")
    print("Compare iFFT with golden vector:")
    error_cnt = 0
    round_factor = 6
    with open("FFT_in.txt", "r") as iFFT_out:
        for i in range(1024):
            golden = float(iFFT_out.readline())
            if round(ifft_out[i], round_factor) != round(golden, round_factor):
                print(f"{i:<4}Failed!! Expect value: {golden:.17f}. Actual value: {ifft_out[i]:.17f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    
    print("===============================")
    print("Exit process")



Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
0b10111111101000
Kernel execution time [mode: NTT]: 0.0002777576446533203 s
0b10010000010000
Kernel execution time [mode: iNTT]: 0.00024318695068359375 s
Kernel execution time [mode: FFT]: 0.0001666545867919922 s
Kernel execution time [mode: iFFT]: 0.00016880035400390625 s
-------------------------------
Compare NTT with golden vector:
Test Passed!!
-------------------------------
Compare iNTT with golden vector:
Test Passed!!
-------------------------------
Compare FFT with golden vector:
Test Passed!!
-------------------------------
Compare iFFT with golden vector:
Test Passed!!
Exit process
